## use follow code to get PPMI data and label

In [ ]:
import numpy as np
import scipy.io as io

y = io.loadmat("D:\\机器学习前沿实验\\实验课一\\dataset\\PPMI.mat")
NC = np.asarray(y["NC"])
NC_label = np.full((NC.shape[0], 1), 0, dtype=int)
PD = np.asarray(y["PD"])
PD_label = np.full((PD.shape[0], 1), 1, dtype=int)


Label_1 = np.vstack((NC_label, PD_label))
Data = np.vstack((NC, PD))
Label = Label_1.reshape(Label_1.shape[0])

print(Data.shape)
print(Label.shape)

## use follow code to get ADNI data and label

In [ ]:
# import scipy.io as io
# import numpy as np
# y=io.loadmat("D:\\机器学习前沿实验\\实验课一\\dataset\\ADNI.mat")
# ##AD
# AD=np.asarray(y['AD'])
# AD_label=np.zeros([AD.shape[0],1],dtype=int)
# ##MCI
# MCI=np.asarray(y['MCI'])
# MCIn=np.asarray(y['MCIn'])
# MCIp=np.asarray(y['MCIp'])
# MCI=np.vstack((MCI,MCIn,MCIp))
# MCI_label=np.ones([MCI.shape[0],1],dtype=int)
# ##NC
# NC=np.asarray(y['NC'])
# NC_label=np.full((NC.shape[0],1),2,dtype=int)

# ##合并:
# print(AD_label.shape)
# # print(EMCI_lable.shape)
# # print(LMCI_lable.shape)
# print(MCI_label.shape)
# print(NC_label.shape)


# Data=np.vstack((AD,MCI,NC))
# Label=np.vstack((AD_label,MCI_label,NC_label))

# print(Data.shape)
# print(Label.shape)

## import necessray library

In [ ]:
import keras.backend as K
import numpy as np
from keras.callbacks import LearningRateScheduler
from keras.layers import LSTM, BatchNormalization, Dense, Dropout
from keras.layers.core import Dense, Dropout, Flatten
from keras.models import Sequential


def scheduler(epoch):
    if epoch == 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 10)
    # 每隔5个epoch，学习率减小为原来的1/10
    if epoch % 10 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr * 10)
        K.set_value(model.optimizer.lr, lr * 0.1)
        print("lr changed to {}".format(lr * 0.1))
    return K.get_value(model.optimizer.lr)


reduce_lr = LearningRateScheduler(scheduler)

## scale data method--我们在论文中提出的按照七种不同缩放方法和四个不同处理方向的预处理策略

In [ ]:
import numpy as np

# #######################
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
)


def ScaleData(train_x, scaling_method, dimension, random_seed):
    if scaling_method == "none":
        return train_x
    if np.isinf(train_x).any():
        print("Train or test set contains infinity.")
        exit(-1)
    scaling_dict = {
        "minmax": MinMaxScaler(),
        "maxabs": MaxAbsScaler(),
        "standard": StandardScaler(),
        "robust": RobustScaler(),
        "quantile": QuantileTransformer(random_state=random_seed),
        "powert": PowerTransformer(),
        "normalize": Normalizer(),
    }
    if scaling_method not in scaling_dict.keys():
        print(f"Scaling method {scaling_method} not found.")
        exit(-1)
    if dimension not in ["timesteps", "channels", "all", "both"]:
        print(f"Dimension {dimension} not found.")
        exit(-1)

    dim1 = -1
    dim2 = 1
    if scaling_method == "normalize":
        dim1 = 1
        dim2 = -1
    out_train_x = np.zeros_like(train_x, dtype=np.float64)

    train_shape = train_x.shape
    if dimension == "all":
        out_train_x = (
            scaling_dict[scaling_method]
            .fit_transform(train_x.reshape((dim1, dim2)))
            .reshape(train_shape)
        )
    else:
        if dimension == "channels":
            train_channel_shape = train_x[:, 0, :].shape
            for i in range(train_x.shape[1]):
                out_train_x[:, i, :] = (
                    scaling_dict[scaling_method]
                    .fit_transform(train_x[:, i, :].reshape((dim1, dim2)))
                    .reshape(train_channel_shape)
                )

        elif dimension == "timesteps":
            train_timest_shape = train_x[:, :, 0].shape

            for i in range(train_x.shape[2]):
                out_train_x[:, :, i] = (
                    scaling_dict[scaling_method]
                    .fit_transform(train_x[:, :, i].reshape((dim1, dim2)))
                    .reshape(train_timest_shape)
                )

        elif dimension == "both":
            train_both_shape = train_x[:, 0, 0].shape
            for i in range(train_x.shape[1]):
                for j in range(train_x.shape[2]):
                    out_train_x[:, i, j] = (
                        scaling_dict[scaling_method]
                        .fit_transform(train_x[:, i, j].reshape((dim1, dim2)))
                        .reshape(train_both_shape)
                    )

        else:
            print(f"Dimension {dimension} not found.")
            exit(-1)
    return out_train_x

# transfer data to three dimension so that we can use scaledata method

In [ ]:
Data = np.expand_dims(Data, axis=-1)
Data = Data.transpose(0, 2, 1)

from tensorflow.keras.utils import to_categorical

Label_new = to_categorical(Label)

In [ ]:
def build_model(dropout_rate=0.25, activation="relu", num_class=2):
    start_neurons = 512
    # create model
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(start_neurons, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    model.add(Dense(start_neurons // 2, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    model.add(Dense(start_neurons // 4, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    model.add(Dense(start_neurons // 8, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate / 2))

    model.add(Dense(num_class, activation="softmax"))
    model.compile(
        optimizer="rmsprop",  # 加速神经网络
        loss="categorical_crossentropy",  # 损失函数
        metrics=["accuracy"],
    )
    return model

## 使用下面的code来完成MLP的 有/无 scaledata 方法的五折交叉对比实验，可以通过调整使用注释处的code来完成选择是否使用scaledata，并输出五折交叉验证结果

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5折交叉验证

i = 1
a = []
X = ScaleData(Data, "quantile", "both", 100)  ##使用scaledata方法
# X=Data ##不使用scaledata方法
for train_index, test_index in kf.split(X, Label_new):
    print("\n{} of kfold {}".format(i, kf.n_splits))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Label_new[train_index], Label_new[test_index]
    model = build_model()
    history = model.fit(
        X_train,
        y_train,
        batch_size=5,
        epochs=300,
        validation_data=(X_test, y_test),
        callbacks=[reduce_lr],
    )
    x = (np.asarray(history.history["accuracy"]),)
    y = np.asarray(history.history["val_accuracy"])
    b = np.asarray(y)
    a.append(b.max())
    i += 1

b = np.asarray(a)
print("五折交叉验证结果为:", b.mean())

## 使用下面的code来完成SVM和ExtraTreesClassifier，GradientBoostingClassifier，RandomForestClassifier的 有/无 scaledata 方法的五折交叉对比实验，可以通过调整使用注释处的code来完成选择是否使用scaledata，并输出五折交叉验证结果

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5折交叉验证

i = 1
a = []

X = ScaleData(Data, "quantile", "both", 100)

X_1 = X.reshape((-1, NC.shape[1]))  ##使用这行代码获取进行scaledata方法后的数据进行训练
# X_1=Data.reshape((-1,NC.shape[1])) ##使用这行代码获取未进行scaledata方法的数据（原始数据）进行训练
for train_index, test_index in kf.split(X_1, Label):
    print("\n{} of kfold {}".format(i, kf.n_splits))
    X_train, X_test = X_1[train_index], X_1[test_index]
    y_train, y_test = Label[train_index], Label[test_index]
    ##  使用下面的代码获取不同模型，包括：梯度提升书，极度提升树（论文所使用），随机深林，
    # from sklearn import ensemble
    # model=ensemble.GradientBoostingClassifier() # 梯度提升树
    # model=ensemble.ExtraTreesClassifier()
    # model=ensemble.RandomForestClassifier()
    ## 使用下面的代码来获取SVM模型(论文所使用)
    from sklearn import svm

    model = svm.SVC()
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(y_test, pred)
    accuracy = (cm[0, 0] + cm[1, 1]) / sum(sum(cm))
    print(accuracy)
    #     print(cm)
    a.append(accuracy)
    i += 1
b = np.asarray(a)
print("五折交叉验证结果为:", b.mean())